![PPGI_UFRJ](imagens/ppgi-ufrj.png)
# Fundamentos de Ciência de Dados

---
[![DOI](https://zenodo.org/badge/335308405.svg)](https://zenodo.org/badge/latestdoi/335308405)

---
# PPGI/UFRJ 2020.3
## Prof Sergio Serra e Jorge Zavaleta

---
# Módulo 4. Deep Learning

## TensorFlow 2.0
> Um **tensor** é um objeto matemático e uma generalização de escalares, vetores e matrizes. Um tensor pode ser representado como uma matriz multidimensional.

> Um tensor de ordem zero (rank) é um escalar. Um vetor / matriz é um tensor de ordem 1, enquanto uma matriz é um tensor de ordem 2. Em consequência, um tensor pode ser considerado uma matriz n-dimensional.

> Exemplos de tensores:
> - 7 : é um tensor de ordem 0. Escalar com shape [].
> - [3.,5, 13]: Tensor de ordem 1. É um vetor com shape [3]
> - [[10,4,6],[3,4,5]]: É um tensor de ordem 2. É uma matriz com shpe [2,3]
> - [[[17,5,1]],[[9,6,3]]]: É um tensor de ordem 3 com shape [2,1,3]

## Grafo Computacional e Session v1.0
> O **tensorflow** tem dois programas básico que formam parte do kernel e executam duas ações principais:
> - Construir um grafo computacional na fase inicial (fase de construção)
> - Rodar ou executar o grafo computacional na fase de execução.

> Um **grafo computacional** é uma série de operações do TensorFlow organizadas em nós de um gráfo.

> Em **TensorFlow**, se pode configurar um grafo (um grafo padrão). Em seguida, se precisa criar variáveis, marcadores de posição e  valores constantes e, em seguida, se cria a sessão e inicializa as variáveis. Finalmente, se alimenta com os dados os marcadores de posição, de modo a invocar qualquer ação.

> Para finalmente avaliar os nós, se deve executar o grafo computacional em uma **sessão**.

## Keras
> **Keras** é uma API de deep Learning escrita em Python, executada em cima de **TensorFlow**. Foi projetada para permitir experimentação rápida em redes neurais profundas. [Keras](https://keras.io/)

> A estrurura de dados básica do Keras são as **camadas** (layer) e os **modelos** (models)

In [ ]:
# modelo sequencial
from tensorflow.keras.models import Sequential
#
model = Sequential()

In [ ]:
# empilhando camadas: add()
from tensorflow.keras.layers import Dense
#
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=10, activation='softmax'))

In [ ]:
# criar o modelo e configurar: .comple()
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

## Tensorflow 2.0

In [ ]:
# 1. importar as bibliotecas
import tensorflow as tf
#
print('tf:',tf.__version__)

> ### Definição de Constantes

In [ ]:
# definição de constantes
c1 = tf.constant(6.3)
c2 = tf.constant(5.0)
c3 = tf.constant(7.0)
print(c1,c2,c3)

>### Operações

In [ ]:
# Operations
r1 = tf.add(c2,c3)                    # soma
print('soma:',r1)
r2 = tf.add(c2,-c3)                   # substração
print('subtração:',r2)
#
op2=tf.multiply(c2,c3)               # multiplicação
print('Multiplicação:',op2)
op2=tf.math.divide(c2,c3)            # divide
print('Divisão:',op2)

In [ ]:
# equação  y = ax + b
a = tf.constant([[10,10],[11.,1.]])
x = tf.constant([[1.,0.],[0.,1.]])
b = tf.Variable(12.)
y = tf.matmul(a, x) + b
print(y)

# Regressão Linear Usando TensorFlow e Keras

In [ ]:
# importar as bilbliotecas
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import seaborn as sb
#import tensorflow as tf
from tensorflow import keras as ks
from tensorflow.estimator import LinearRegressor
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# dataset boston
boston_load = datasets.load_boston()
feature_columns = boston_load.feature_names                  
boston_data = pd.DataFrame(boston_load.data,columns=feature_columns).astype(np.float32)
boston_data.head()

In [ ]:
# target column
target_column = boston_load.target
print(target_column)

In [ ]:
boston_data['MEDV'] = target_column.astype(np.float32)
boston_data.head()

In [ ]:
# 3. relações entre variaveis
sb.pairplot(boston_data, diag_kind="kde");

In [ ]:
# Correlação
correlation_data = boston_data.corr()
correlation_data.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
# Estatistica descitiva
stats = boston_data.describe()
boston_stats = stats.transpose()
boston_stats

In [ ]:
# Selecionar colunas requeridas 
X_data = boston_data[[i for i in boston_data.columns if i not in ['MEDV']]]
Y_data = boston_data[['MEDV']]

In [ ]:
# Treinamento e teste
training_features , test_features ,training_labels, test_labels = train_test_split(X_data , Y_data , test_size=0.2)

In [ ]:
print('No. de filas no Treinamento: ', training_features.shape[0])
print('No. de filas no Teste: ', test_features.shape[0])
print()
print('No. de colunas no Treinamento: ', training_features.shape[1])
print('No. de colunas no Teste: ', test_features.shape[1])
print()
print('No. de filas nas rótulos de Treinamento: ', training_labels.shape[0])
print('No. de filas nos rótulos de Teste: ', test_labels.shape[0])
print()
print('No. de colunas nos rótulos de Treinamento: ', training_labels.shape[1])
print('No. de colunas nos rótulos de Teste: ', test_labels.shape[1])

In [ ]:
# Normalizado os dados
def norm(x):
    stats = x.describe()
    stats = stats.transpose()
    return (x - stats['mean']) / stats['std']

In [ ]:
# normaliza
normed_train_features = norm(training_features)
print(normed_train_features)
normed_test_features = norm(test_features)
#print(normed_test_features)

In [ ]:
# Contruir o pipeline para modelo do TensorFlow
def feed_input(features_dataframe, target_dataframe,num_of_epochs=10, shuffle=True, batch_size=32):
    #
    def input_feed_function():
        dataset = tf.data.Dataset.from_tensor_slices((dict(features_dataframe), target_dataframe))
        if shuffle:
            dataset = dataset.shuffle(2000)
        dataset = dataset.batch(batch_size).repeat(num_of_epochs)
        return dataset
    #
    return input_feed_function

In [ ]:
# Usando a função definida acima
train_feed_input = feed_input(normed_train_features,training_labels)
train_feed_input_testing = feed_input(normed_train_features,training_labels, num_of_epochs=1, shuffle=False)
test_feed_input = feed_input(normed_test_features,test_labels, num_of_epochs=1, shuffle=False)

In [ ]:
# Modelando o treiamento
feature_columns_numeric = [tf.feature_column.numeric_column(m) for m in training_features.columns]
# modelo
linear_model = LinearRegressor(feature_columns=feature_columns_numeric, optimizer='RMSProp')
#
linear_model.train(train_feed_input)

In [ ]:
# predições
train_predictions = linear_model.predict(train_feed_input_testing)
test_predictions = linear_model.predict(test_feed_input)
train_predictions_series = pd.Series([p['predictions'][0] for p in train_predictions])
test_predictions_series = pd.Series([p['predictions'][0] for p in test_predictions])

In [ ]:
train_predictions_df = pd.DataFrame(train_predictions_series, columns=['predictions'])
test_predictions_df = pd.DataFrame(test_predictions_series, columns=['predictions'])
#
training_labels.reset_index(drop=True, inplace=True)
train_predictions_df.reset_index(drop=True, inplace=True)
#
test_labels.reset_index(drop=True, inplace=True)
test_predictions_df.reset_index(drop=True, inplace=True)
#
train_labels_with_predictions_df = pd.concat([training_labels, train_predictions_df], axis=1)
test_labels_with_predictions_df = pd.concat([test_labels,test_predictions_df], axis=1)

In [ ]:
# validação
def calculate_errors_and_r2(y_true, y_pred):
    mean_squared_err = (mean_squared_error(y_true, y_pred))
    root_mean_squared_err = np.sqrt(mean_squared_err)
    r2 = round(r2_score(y_true, y_pred)*100,0)
    return mean_squared_err, root_mean_squared_err, r2
#
train_mean_squared_error, train_root_mean_squared_error,train_r2_score_percentage = calculate_errors_and_r2(training_labels, train_predictions_series)
#
test_mean_squared_error, test_root_mean_squared_error,test_r2_score_percentage = calculate_errors_and_r2(test_labels, test_predictions_series)
#
print('Dados de Treinamento - Mean Squared Error = ', train_mean_squared_error)
print('Dados de Treinamento - Root Mean Squared Error = ', train_root_mean_squared_error)
print('Dados de Treinamento - R2 = ', train_r2_score_percentage)
print('Dado de Teste - Mean Squared Error = ', test_mean_squared_error)
print('Dados de Teste - Root Mean Squared Error = ', test_root_mean_squared_error)
print('Dados de Teste - R2 = ', test_r2_score_percentage)

## Contsruir uma Rede Neural com TensorFlow
> Dataset **MNIST dataset** [Dataset](http://yann.lecun.com/exdb/mnist/)

In [ ]:
# Load Dataset usando o Keras
(training_images, training_labels), (test_images, test_labels) = ks.datasets.fashion_mnist.load_data()

In [ ]:
# visualizando alguns dados
print('Imagenes de Treinamento - Dataset Shape: {}'.format(training_images.shape))
print('No. de imagens de Treinamento - Dataset Labels: {}'.format(len(training_labels)))
print('Imagens de teste - Dataset Shape: {}'.format(test_images.shape))
print('No. de imagens de Teste - Dataset Labels: {}'.format(len(test_labels)))

In [ ]:
# re-escala (0-255) -> 0-1
training_images = training_images / 255.0
test_images = test_images / 255.0

In [ ]:
# usando keras para as diferentes camadas
input_data_shape = (28, 28)                                # entrada de 28x28 pixels
#
hidden_activation_function = 'relu'                        # função de ativação - camada oculta
output_activation_function = 'softmax'                     # função de ativação  - camada de saida
#
nn_model = ks.Sequential()                                 # modelo sequencial
#
nn_model.add(ks.layers.Flatten(input_shape=input_data_shape, name='Input_layer'))
nn_model.add(ks.layers.Dense(32, activation=hidden_activation_function, name='Hidden_layer'))
nn_model.add(ks.layers.Dense(10, activation=output_activation_function, name='Output_layer'))
nn_model.summary()

In [ ]:
# otimizando : SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax, Nadam, Ftrl
#
optimizer = 'adam'
loss_function = 'sparse_categorical_crossentropy'
metric = ['accuracy']
#
nn_model.compile(optimizer=optimizer, loss=loss_function, metrics=metric)    # compilar
#
nn_model.fit(training_images, training_labels, epochs=10)                    # ajustar ao modelo

In [ ]:
# Avaliação do treinamento
training_loss, training_accuracy = nn_model.evaluate(training_images, training_labels)
print('Acurácia dos dados de Treinamento {}'.format(round(float(training_accuracy),2)*100),'%')

In [ ]:
# Avaliação do teste
test_loss, test_accuracy = nn_model.evaluate(test_images,test_labels)
print('Acurácia dos dados de Teste {}'.format(round(float(test_accuracy),2)*100),'%')

## Usando o tensorboard

In [ ]:
%load_ext tensorboard
import datetime, os

In [ ]:
# dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
#
(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
#
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
# criando modelo simples
def create_model():
    #
    return tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation='softmax')
            ])

In [ ]:
# treinamento do modelo
def train_model():
    #
    model = create_model()
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    #
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    #
    model.fit(x=x_train, y=y_train, epochs=5, 
            validation_data=(x_test, y_test), 
            callbacks=[tensorboard_callback])

In [ ]:
train_model()

In [ ]:
# iniciar o tensorboar
%tensorboard --logdir logs

In [ ]:
# %tensorboard --logdir logs usar antes do treinamento
#!kill 8104

---
#### Fundamentos para Ciência Dados &copy; Copyright 2021, Sergio Serra & Jorge Zavaleta